![](https://www.italianowine.com/copia/wp-content/uploads/2018/09/Classificazione-italia-europa-EN.jpg)italienowine.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import matplotlib.style as style
import plotly.express as px
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadswinecsv/wine.csv', delimiter=';', encoding = "utf8", nrows = nRowsRead)
df.dataframeName = 'wine.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

#Codes by  Rishabh Raj Shukla   https://www.kaggle.com/rajshukla1102/beginnernotebook/notebook

In [ ]:
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df.plot(subplots=True, sharex=True ,figsize=(20,50))
plt.show()

In [ ]:
df.drop('quality', axis=1).corrwith(df.quality).plot(kind='bar', grid=True, figsize=(12, 8), color=['salmon'],
                                                   title="Correlation with Quality")

In [ ]:
dataset = pd.get_dummies(df, columns=['alcohol'], drop_first=True)

In [ ]:
sc = StandardScaler()
scale_var = ['citric_acid', 'pH', 'sulphates', 'free_sulfur_dioxide', 'residual_sugar']
dataset[scale_var] = sc.fit_transform(dataset[scale_var])

In [ ]:
X = dataset.drop('quality', axis=1)
y = dataset.quality

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
allAlgo = [('lr', LogisticRegression()),('knn', KNeighborsClassifier()),('dclf', DecisionTreeClassifier()),
          ('svm', SVC()),('nb', GaussianNB())]

In [ ]:
res = []
names = []
for name, algo in allAlgo:
    kfold = KFold(n_splits=10, random_state=None)
    cv_results = cross_val_score(algo, X_train, y_train, cv=kfold, scoring='accuracy')
    res.append(cv_results)
    names.append(name)
    print(name, cv_results.mean(), cv_results.std())

In [ ]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n**********************************************")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n************************************************")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
lr = LogisticRegression(solver='saga',penalty='elasticnet',l1_ratio=0.6,max_iter=1000)
lr.fit(X_train, y_train)

print_score(lr, X_train, y_train, X_test, y_test, train=True)
print_score(lr, X_train, y_train, X_test, y_test, train=False)

#Applying PCA on Logistic Regression

In [ ]:
#Applying PCA on Logistic Regression

pca = PCA(n_components=10) 
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
expained_variance = pca.explained_variance_ratio_
classifier = LogisticRegression(random_state = 1)
classifier.fit(X_train_pca, y_train)
print_score(classifier, X_train_pca, y_train, X_test_pca, y_test, train=True)
print_score(classifier, X_train_pca, y_train, X_test_pca, y_test, train=False)

#Support Vector Machine

In [ ]:
sv = SVC(kernel='rbf',C=1)
sv.fit(X_train, y_train)
print_score(sv, X_train, y_train, X_test, y_test, train=True)
print_score(sv, X_train, y_train, X_test, y_test, train=False)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print_score(knn, X_train, y_train, X_test, y_test, train=True)
print_score(knn, X_train, y_train, X_test, y_test, train=False)

In [ ]:
gn = GaussianNB()
gn.fit(X_train, y_train)
print_score(gn, X_train, y_train, X_test, y_test, train=True)
print_score(gn, X_train, y_train, X_test, y_test, train=False)

In [ ]:
#DecisionTreeClassifier

dc = DecisionTreeClassifier()
dc.fit(X_train, y_train)
print_score(dc, X_train, y_train, X_test, y_test, train=True)
print_score(dc, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_validation = cross_val_score(estimator = lr, X = X_train, y = y_train, cv = 10)
print("Cross validation of LR model = ",cross_validation)
print("Cross validation of LR model (in mean) = ",cross_validation.mean())

#The coef_ did not converge", ConvergenceWarning)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_validation = cross_val_score(estimator = sv, X = X_train, y = y_train, cv = 10)
print("Cross validation of SVC model = ",cross_validation)
print("Cross validation of SVC model (in mean) = ",cross_validation.mean())

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thanks Rishabh Raj Shukla for the script' )